# Head

In [7]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%pip install causal_conv1d datasets

# Lab

In [12]:
import torch, InitMamba, importlib
importlib.reload(InitMamba)
import VAE
importlib.reload(VAE)
from VAE import MambaVAE
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
tokenizer.add_special_tokens({"bos_token": "<|startoftext|>"})
vae = MambaVAE().cuda().eval()
vae.load_state_dict(torch.load('./results/CoT_vae/model.pth', weights_only=True))

<All keys matched successfully>

In [39]:
inputs1 = tokenizer(["<|startoftext|>According to the given context, as temperature increases particles move faster. Tina's body is at a colder temperature than before, so we can infer that there will be less energy in her body and thus less movement of particles. So slower would be the answer because it matches with our inference from the temperatures mentioned above Answer: slower<|endoftext|>"], return_tensors='pt').to('cuda')
inputs2 = tokenizer(['<|startoftext|>The word "sad" is repeated multiple times in the text and it is also combined with other words indicating sadness like "late", "birthday". Hence, it can be inferred that the underlying emotion of the text is sad. Answer: sad<|endoftext|>'], return_tensors='pt').to('cuda')

In [40]:
inputs1['input_ids'].shape

torch.Size([1, 67])

In [ ]:
res1 = vae(**inputs1)
print(res1[:2])
print(tokenizer.batch_decode(res1[2].argmax(-1)))

res2 = vae(**inputs2)
print(res2[:2])
print(tokenizer.batch_decode(res2[2].argmax(-1)))

In [ ]:
r = 0
noise = torch.randn_like(res1[3])
mix_state = (res1[3]*r+res2[3]*(1-r))
tokens = vae.decoder.generate(**tokenizer('<|startoftext|>', return_tensors='pt').to('cuda'),
                              inputs_ssm_states = mix_state,
                              max_length = 128,
                              do_sample=True,
                              num_return_sequences=3)
tokenizer.batch_decode(tokens, skip_special_tokens=True)

# Eval


In [11]:
import torch, transformers, os
from transformers import TrainingArguments, AutoTokenizer
from InitMamba import MambaForCausalLM
from datasets import load_dataset
import Dataset

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
model = MambaForCausalLM.from_pretrained('state-spaces/mamba-130m-hf').cuda()
dataset = load_dataset("arrow", data_files = './cache/CoT_full.arrow', split = 'train').select(range(100000))
print(dataset)
tot = len(dataset)
eval_size= int(tot * 0.05)
train_dataset = dataset.select(range(eval_size, tot))
eval_dataset = dataset.select(range(eval_size))
model.load_state_dict(torch.load("results/CoT_full/model.pth", weights_only=True))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


Generating train split: 0 examples [00:00, ? examples/s]

KeyboardInterrupt: 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
input_str = tokenizer.decode(eval_dataset[1]['input_ids'], skip_special_tokens=True)
input_ids = tokenizer(input_str.split(' Thinking: ')[0] + ' Thinking: ', return_tensors = 'pt')['input_ids'].cuda()
tokens = model.generate(
    input_ids,
    max_length = 512,
    # num_return_sequences = 5,
    # do_sample = False,
    # num_beams = 5
    )
print(input_str)
tokenizer.batch_decode(tokens)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The 'batch_size' argument of MambaCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.


Choose your reply from the options at the end. Does "The Tucson-Pima County Bicycle Advisory Committee (TPCBAC) serves in an advisory capacity to local governments on issues relating to bicycle recreation, transportation, and safety." contain the correct answer to "What organization advises the Tucson government on bike concerns?"
OPTIONS:
- yes
- no Thinking: The passage talks about the Tucson-Pima County Bicycle Advisory Committee serving in an advisory capacity to local governments on issues relating to bicycle recreation, transportation and safety. So, it does contain the correct answer for what organization advises the Tucson government on bike concerns. Hence, yes is our choice of reply Answer: yes


['Choose your reply from the options at the end. Does "The Tucson-Pima County Bicycle Advisory Committee (TPCBAC) serves in an advisory capacity to local governments on issues relating to bicycle recreation, transportation, and safety." contain the correct answer to "What organization advises the Tucson government on bike concerns?"\nOPTIONS:\n- yes\n- no Thinking: \nThe passage talks about the Tucson-Pima County Bicycle Advisory Committee (TPCBAC) serving in an advisory capacity to local governments on issues relating to bicycle recreation, transportation and safety. It does not mention anything about bike concerns or whether it advises the government on these issues. So, the answer is "no". Answer: no<|endoftext|>']

# Run

In [10]:
!python Trainer.py

E0000 00:00:1739729601.046537    8108 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739729601.060090    8108 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1466151
})
The 'batch_size' argument of MambaCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' a

In [ ]:
!python Dataset.py

In [ ]:
!python CoT_full_trainer.py

In [ ]:
from google.colab import runtime
runtime.unassign()